In [166]:
# Evaluate
import os
import plotly.graph_objects as go
import trimesh, numpy as np
import torch
from points_dataset import EmbedderDataset
from models import DiffusionNetAutoencoder
from point_cloud_datasest import PointCloudDataset

num_points = 256
num_points = 1024
stl_case_path1 = r"E:\awsCollectedDataPeydroNew_2024\stl\1700129\34.stl" # 11
stl_case_path2 = r"E:\awsCollectedDataPeydroNew_2024\stl\1700129\16.stl"

tooth_mesh1 = trimesh.load_mesh(stl_case_path1)
tooth_mesh2 = trimesh.load_mesh(stl_case_path2)
# original points
pointsOrig1, _ = trimesh.sample.sample_surface_even(tooth_mesh1, count = num_points)
pointsOrig2, _ = trimesh.sample.sample_surface_even(tooth_mesh2, count = num_points)


In [167]:
def plot_points(points, epoch=None):
    # Create a 3D scatter plot
    fig = go.Figure(data=[go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(
            size=1,
            color='blue',  # Color of points
        )
    )])

    # Update layout for better visualization
    fig.update_layout(scene=dict(
                        xaxis_title='X',
                        yaxis_title='Y',
                        zaxis_title='Z'),
                    width=800,
                    margin=dict(r=20, l=10, b=10, t=10))
    if epoch:
        fig.write_html(f"img/predicted_shape_epoch_{epoch}.html")
        # fig.write_image(f"img/predicted_shape_epoch_{epoch}.jpg")

    fig.show()

In [168]:
# predict points
# Initialize model
POINT_DIM = 3 
hidden_features = 64
latent_dim = 32

model_epoch = 194
model = DiffusionNetAutoencoder(POINT_DIM, hidden_features, latent_dim)
model.load_state_dict(torch.load(f'models/best_model_epoch_{model_epoch}.pth'))
model.eval().to("cpu")

points_to_pred = torch.from_numpy(np.array(pointsOrig1, dtype=np.float32))
# points_to_pred += 10
batch = PointCloudDataset((points_to_pred,), k=6)
pred, hidden = model(batch[0].x, batch[0].edge_index, batch[0].edge_weight)
points = pred.detach().numpy()
# points = points_to_pred
points.shape # (1024, 3)

plot_points(points, 
            # epoch=model_epoch # save html
            )
plot_points(points_to_pred)
pred.shape, hidden.shape # (torch.Size([256, 3]), torch.Size([256, 32])) ??????? 256 = num_points

C:\Users\5010858\AppData\Local\Temp/ipykernel_29848/1744016969.py:9: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



(torch.Size([1024, 3]), torch.Size([1024, 32]))